In [1]:
# Load libraries

# # Plotting utils 
import matplotlib.pyplot as plt 
import matplotlib.colors as colors
import matplotlib.ticker as ticker 
import matplotlib.patches as patches
import matplotlib as matplotlib
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import make_axes_locatable

# import seaborn as sns

# Analysis
import time
import numpy as np 
import xarray as xr
import datetime
from   datetime import date, timedelta
import pandas as pd 
import pickle
import scipy.stats as stats
import glob
import os 

import metpy.calc as mpc
from metpy.units import units
from metpy.plots import Hodograph, SkewT

import warnings
warnings.filterwarnings('ignore')

## Import Ngl with pyn_env active 
import Ngl



**Useful functions**

In [2]:
# Grabbed from Brian M. to use time midpoints, not end periods
def cesm_correct_time(ds):
    """Given a Dataset, check for time_bnds,
       and use avg(time_bnds) to replace the time coordinate.
       Purpose is to center the timestamp on the averaging inverval.   
       NOTE: ds should have been loaded using `decode_times=False`
    """
    assert 'time_bnds' in ds
    assert 'time' in ds
    correct_time_values = ds['time_bnds'].mean(dim='nbnd')
    # copy any metadata:
    correct_time_values.attrs = ds['time'].attrs
    ds = ds.assign_coords({"time": correct_time_values})
    ds = xr.decode_cf(ds)  # decode to datetime objects
    return ds


In [3]:
'''
Define a function to get the height of the PBL as the level with maximum d(var)/dz. 
Inputs:  A dataset with CAM output ('DS') and the variable to differentiate ('pbl_var')
Outputs: An array with boundary layer depth
'''
def PBLHasMaxDZ(DS, pbl_var): 
    # Convert HMGcamDS_all to height (nabbed from Rich's script)
    p0 = DS['P0'].values[0]
    
    plevm = DS['hyam']*p0 + DS['hybm']*DS['PS'].isel(lat=0,lon=0) # Mid level
    plevm.attrs['units'] = "Pa"

    # Height with standard atmosphere
    zlevm      = plevm
    zlevm_vals = 1000.*np.asarray(mpc.pressure_to_height_std(plevm)) # Units of [m] after multiplied 
    zlevm      = plevm.copy(deep=True)
    zlevm[:,:] = zlevm_vals
    
    pvar        = DS[pbl_var].isel(lat=0,lon=0)
    pvar['lev'] = zlevm[0,:].values
    dvardz      = pvar.differentiate("lev") # Find field gradient wrt HEIGHT!

    dvardz.loc[:,200:]   = 0.  # Restrict to a specificheight region
    dvardz.loc[:,:3000.] = 0

    nT = np.shape(dvardz)[0]
    PBLdepth = np.full([nT], np.nan)

    for iT in range(nT):
        iLevs  = np.where((zlevm[iT,:]>=200) & (zlevm[iT,:]<=3000))[0]
        maxLev = np.where(dvardz[iT,iLevs]==np.nanmax(dvardz[iT,iLevs]))[0]
        PBLdepth[iT] = zlevm[iT,iLevs[maxLev[0]]]
    
    return PBLdepth

In [4]:
'''
Define a function to get the height of the PBL as the level with maximum d(var)/dz. 
Inputs:  A dataset with CAM output ('DS') and the variable to differentiate ('pbl_var')
Outputs: An array with boundary layer depth
'''
def PBLHasMaxDZ_abs(DS, pbl_var): 
    # Convert HMGcamDS_all to height (nabbed from Rich's script)
    p0 = DS['P0'].values[0]
    
    plevm = DS['hyam']*p0 + DS['hybm']*DS['PS'].isel(lat=0,lon=0) # Mid level
    plevm.attrs['units'] = "Pa"

    # Height with standard atmosphere
    zlevm      = plevm
    zlevm_vals = 1000.*np.asarray(mpc.pressure_to_height_std(plevm)) # Units of [m] after multiplied 
    zlevm      = plevm.copy(deep=True)
    zlevm[:,:] = zlevm_vals
    
    pvar        = DS[pbl_var].isel(lat=0,lon=0)
    pvar['lev'] = zlevm[0,:].values
    dvardz      = pvar.differentiate("lev") # Find field gradient wrt HEIGHT!

    dvardz.loc[:,200:]   = 0.  # Restrict to a specificheight region
    dvardz.loc[:,:3000.] = 0

    nT = np.shape(dvardz)[0]
    PBLdepth = np.full([nT], np.nan)

    for iT in range(nT):
        iLevs  = np.where((zlevm[iT,:]>=200) & (zlevm[iT,:]<=3000))[0]
        maxLev = np.where(np.abs(dvardz[iT,iLevs])==np.nanmax(np.abs(dvardz[iT,iLevs])))[0]
        PBLdepth[iT] = zlevm[iT,iLevs[maxLev[0]]]
    
    return PBLdepth

In [5]:
def interpolateToPressure_v2(DS, varName, pressGoals):
#     nCases = len(DSin.case.values)
#     nTimes = len(DSin.time.values)
    
#     saveOut = np.full([nTimes,len(pressGoals),1,1], np.nan)

    ## For the larger arrays, need to operate case-by-case; input to vinth2p can only be 3 or 4 dimensions. 
#     for iCase in range(nCases): 
#     DS = DSin

    p0mb = DS.P0.values[0]/100        # mb

    # Pull out hya/hyb profiles 
    hyam = DS.hyam.values[0,:]
    hybm = DS.hybm.values[0,:]
    hyai = DS.hyai.values[0,:]
    hybi = DS.hybi.values[0,:]

    # Surface pressure with time dimension
    PS   = DS.PS.values              # Pa 

    # Converting variables: 
    if np.shape(DS[varName].values)[1]==len(DS.ilev.values):
        varInterp = Ngl.vinth2p(DS[varName].values,hyai,hybi,pressGoals,PS,1,p0mb,1,True)
    elif np.shape(DS[varName].values)[1]==len(DS.lev.values):
        varInterp = Ngl.vinth2p(DS[varName].values,hyam,hybm,pressGoals,PS,1,p0mb,1,True)

    saveOut = varInterp
    
    return saveOut


## Read in data

In [6]:
## Where files are saved + start of casename
# dataDir = '/glade/scratch/mdfowler/CLASP_ensOutput/'
dataDir = '/glade/scratch/mdfowler/CLASP_ensOutput/multiplier/mult100_2017/'

htgFileStart = 'FSCAM.T42_T42.CLASP_fullycoupled_FinalOption.onlyThlRt.HTGens'
# hmgFileStart = 'FSCAM.T42_T42.CLASP_fullycoupled_FinalOption.onlyThlRt.HMGens'

## Extra vars I want, but they're on the h0 files instead of the h1 files (same time step, just different stream) 
addVars = np.asarray(['SWCF','LWCF','TS','CLOUD','FSNS','FLNS','PS','QREFHT',
                      'U10','CLDHGH','CLDLIQ','TMQ'])

## Other settings to build up case names 
pertVals = np.asarray(['00'])


pertCount = 0
for iPert in range(len(pertVals)): 
    
    ## **** IMPORTANT: I've added the 2016 option to the listed files to look at just one year right now
    
    # listFilesHMG_camh1 = np.sort(glob.glob(dataDir+hmgFileStart+'*pert'+pertVals[iPert]+'*cam.h1*'))
    listFilesHTG_camh1 = np.sort(glob.glob(dataDir+htgFileStart+'*pert'+pertVals[iPert]+'*cam.h1*'))
    
    # listFilesHMG_camh0 = np.sort(glob.glob(dataDir+hmgFileStart+'*pert'+pertVals[iPert]+'*cam.h0*'))
    listFilesHTG_camh0 = np.sort(glob.glob(dataDir+htgFileStart+'*pert'+pertVals[iPert]+'*cam.h0*'))
    
    # listFilesHMG_clmh0 = np.sort(glob.glob(dataDir+hmgFileStart+'*pert'+pertVals[iPert]+'*clm2.h0.*'))
    listFilesHTG_clmh0 = np.sort(glob.glob(dataDir+htgFileStart+'*pert'+pertVals[iPert]+'*clm2.h0.*'))
    
    fileCount=0
    for iFile in range(len(listFilesHTG_camh1)):
        # with xr.open_dataset(listFilesHMG_camh1[iFile], decode_times=False) as hmgDS: 
        #     hmgDS         = cesm_correct_time(hmgDS)
        #     hmgDS['time'] = hmgDS.indexes['time'].to_datetimeindex() 
        with xr.open_dataset(listFilesHTG_camh1[iFile], decode_times=False) as htgDS: 
            htgDS         = cesm_correct_time(htgDS)
            htgDS['time'] = htgDS.indexes['time'].to_datetimeindex()

        # with xr.open_dataset(listFilesHMG_clmh0[iFile], decode_times=True) as clm_hmgDS: 
        #     clm_hmgDS['time'] = hmgDS['time']
        with xr.open_dataset(listFilesHTG_clmh0[iFile], decode_times=True) as clm_htgDS: 
            clm_htgDS['time'] = htgDS['time']    
            
        
        ## Add variables as needed (some are on h0 file stream)
        for iVar in range(len(addVars)):
            # varHMG         = xr.open_dataset(listFilesHMG_camh0[iFile])[addVars[iVar]]
            # varHMG['time'] = hmgDS['time']

            varHTG         = xr.open_dataset(listFilesHTG_camh0[iFile])[addVars[iVar]]
            varHTG['time'] = htgDS['time']

            # hmgDS = xr.merge([hmgDS, varHMG])
            htgDS = xr.merge([htgDS, varHTG])

        iTimeStart_day1  = np.where(  htgDS.time.values <= (htgDS.time.values[0] + np.timedelta64(1,'D')) )[0]
        iTimeStart_day2  = np.where( (htgDS.time.values >= (htgDS.time.values[0] + np.timedelta64(1,'D'))) & 
                                     (htgDS.time.values <= (htgDS.time.values[0] + np.timedelta64(2,'D'))))[0]
        iTimeStart_day3  = np.where( (htgDS.time.values >= (htgDS.time.values[0] + np.timedelta64(2,'D'))) & 
                                     (htgDS.time.values <= (htgDS.time.values[0] + np.timedelta64(3,'D'))))[0]
        iTimeStart_day4  = np.where( (htgDS.time.values >= (htgDS.time.values[0] + np.timedelta64(3,'D'))) & 
                                     (htgDS.time.values <= (htgDS.time.values[0] + np.timedelta64(4,'D'))))[0]
        iTimeStart_day5  = np.where( (htgDS.time.values >= (htgDS.time.values[0] + np.timedelta64(4,'D'))) & 
                                     (htgDS.time.values <= (htgDS.time.values[0] + np.timedelta64(5,'D'))))[0]
        iTimeStart_day6  = np.where( (htgDS.time.values >= (htgDS.time.values[0] + np.timedelta64(5,'D'))) & 
                                     (htgDS.time.values <= (htgDS.time.values[0] + np.timedelta64(6,'D'))))[0]
        

        # Build larger array 
        if fileCount==0:
#             hmg_allDay1      = hmgDS.isel(time=iTimeStart_day1)
#             hmg_allDay2      = hmgDS.isel(time=iTimeStart_day2)
#             hmg_allDay3      = hmgDS.isel(time=iTimeStart_day3)
#             hmg_allDay4      = hmgDS.isel(time=iTimeStart_day4)
#             hmg_allDay5      = hmgDS.isel(time=iTimeStart_day5)
#             hmg_allDay6      = hmgDS.isel(time=iTimeStart_day6)
            
            # htg_allDay1      = htgDS.isel(time=iTimeStart_day1)
            htg_allDay2      = htgDS.isel(time=iTimeStart_day2)
            htg_allDay3      = htgDS.isel(time=iTimeStart_day3)
            htg_allDay4      = htgDS.isel(time=iTimeStart_day4)
            htg_allDay5      = htgDS.isel(time=iTimeStart_day5)
            htg_allDay6      = htgDS.isel(time=iTimeStart_day6)
            
            # clmHMG_allDay1   = clm_hmgDS.isel(time=iTimeStart_day1)
#             clmHMG_allDay2   = clm_hmgDS.isel(time=iTimeStart_day2)
#             clmHMG_allDay3   = clm_hmgDS.isel(time=iTimeStart_day3)
#             clmHMG_allDay4   = clm_hmgDS.isel(time=iTimeStart_day4)
#             clmHMG_allDay5   = clm_hmgDS.isel(time=iTimeStart_day5)
#             clmHMG_allDay6   = clm_hmgDS.isel(time=iTimeStart_day6)
        
            # clmHTG_allDay1   = clm_htgDS.isel(time=iTimeStart_day1)
            clmHTG_allDay2   = clm_htgDS.isel(time=iTimeStart_day2)
            clmHTG_allDay3   = clm_htgDS.isel(time=iTimeStart_day3)
            clmHTG_allDay4   = clm_htgDS.isel(time=iTimeStart_day4)
            clmHTG_allDay5   = clm_htgDS.isel(time=iTimeStart_day5)
            clmHTG_allDay6   = clm_htgDS.isel(time=iTimeStart_day6)
            
        else: 
#             hmg_allDay1      = xr.concat([hmg_allDay1,    hmgDS.isel(time=iTimeStart_day1)],  dim='time')
#             hmg_allDay2      = xr.concat([hmg_allDay2,    hmgDS.isel(time=iTimeStart_day2)],  dim='time')
#             hmg_allDay3      = xr.concat([hmg_allDay3,    hmgDS.isel(time=iTimeStart_day3)],  dim='time')
#             hmg_allDay4      = xr.concat([hmg_allDay4,    hmgDS.isel(time=iTimeStart_day4)],  dim='time')
#             hmg_allDay5      = xr.concat([hmg_allDay5,    hmgDS.isel(time=iTimeStart_day5)],  dim='time')
#             hmg_allDay6      = xr.concat([hmg_allDay6,    hmgDS.isel(time=iTimeStart_day6)],  dim='time')
            
#             htg_allDay1      = xr.concat([htg_allDay1,    htgDS.isel(time=iTimeStart_day1)],  dim='time')
            htg_allDay2      = xr.concat([htg_allDay2,    htgDS.isel(time=iTimeStart_day2)],  dim='time')
            htg_allDay3      = xr.concat([htg_allDay3,    htgDS.isel(time=iTimeStart_day3)],  dim='time')
            htg_allDay4      = xr.concat([htg_allDay4,    htgDS.isel(time=iTimeStart_day4)],  dim='time')
            htg_allDay5      = xr.concat([htg_allDay5,    htgDS.isel(time=iTimeStart_day5)],  dim='time')
            htg_allDay6      = xr.concat([htg_allDay6,    htgDS.isel(time=iTimeStart_day6)],  dim='time')   
            
            
#             clmHMG_allDay1   = xr.concat([clmHMG_allDay1, clm_hmgDS.isel(time=iTimeStart_day1)], dim='time', data_vars='minimal')
#             clmHMG_allDay2   = xr.concat([clmHMG_allDay2, clm_hmgDS.isel(time=iTimeStart_day2)], dim='time', data_vars='minimal')
#             clmHMG_allDay3   = xr.concat([clmHMG_allDay3, clm_hmgDS.isel(time=iTimeStart_day3)], dim='time', data_vars='minimal')
#             clmHMG_allDay4   = xr.concat([clmHMG_allDay4, clm_hmgDS.isel(time=iTimeStart_day4)], dim='time', data_vars='minimal')
#             clmHMG_allDay5   = xr.concat([clmHMG_allDay5, clm_hmgDS.isel(time=iTimeStart_day5)], dim='time', data_vars='minimal')
#             clmHMG_allDay6   = xr.concat([clmHMG_allDay6, clm_hmgDS.isel(time=iTimeStart_day6)], dim='time', data_vars='minimal')
            
#             clmHTG_allDay1   = xr.concat([clmHTG_allDay1, clm_htgDS.isel(time=iTimeStart_day1)], dim='time', data_vars='minimal')
            clmHTG_allDay2   = xr.concat([clmHTG_allDay2, clm_htgDS.isel(time=iTimeStart_day2)], dim='time', data_vars='minimal')
            clmHTG_allDay3   = xr.concat([clmHTG_allDay3, clm_htgDS.isel(time=iTimeStart_day3)], dim='time', data_vars='minimal')
            clmHTG_allDay4   = xr.concat([clmHTG_allDay4, clm_htgDS.isel(time=iTimeStart_day4)], dim='time', data_vars='minimal')
            clmHTG_allDay5   = xr.concat([clmHTG_allDay5, clm_htgDS.isel(time=iTimeStart_day5)], dim='time', data_vars='minimal')
            clmHTG_allDay6   = xr.concat([clmHTG_allDay6, clm_htgDS.isel(time=iTimeStart_day6)], dim='time', data_vars='minimal')
            
        fileCount = fileCount+1
        print('Done with day %i of %i ' % (iFile, len(listFilesHTG_camh1)) )
            
    ## Combine into larger HTG or HMG arrays for all pertlim experiments 
#     hmg_allDay1  = hmg_allDay1.assign_coords({"pertlim": iPert})
#     hmg_allDay2  = hmg_allDay2.assign_coords({"pertlim": iPert})
#     hmg_allDay3  = hmg_allDay3.assign_coords({"pertlim": iPert})
#     hmg_allDay4  = hmg_allDay4.assign_coords({"pertlim": iPert})
#     hmg_allDay5  = hmg_allDay5.assign_coords({"pertlim": iPert})
#     hmg_allDay6  = hmg_allDay6.assign_coords({"pertlim": iPert})
    
#     htg_allDay1  = htg_allDay1.assign_coords({"pertlim": iPert})
    htg_allDay2  = htg_allDay2.assign_coords({"pertlim": iPert})
    htg_allDay3  = htg_allDay3.assign_coords({"pertlim": iPert})
    htg_allDay4  = htg_allDay4.assign_coords({"pertlim": iPert})
    htg_allDay5  = htg_allDay5.assign_coords({"pertlim": iPert})
    htg_allDay6  = htg_allDay6.assign_coords({"pertlim": iPert})
    
#     clmHMG_allDay1 = clmHMG_allDay1.assign_coords({"pertlim": iPert})
#     clmHMG_allDay2 = clmHMG_allDay2.assign_coords({"pertlim": iPert})
#     clmHMG_allDay3 = clmHMG_allDay3.assign_coords({"pertlim": iPert})
#     clmHMG_allDay4 = clmHMG_allDay4.assign_coords({"pertlim": iPert})
#     clmHMG_allDay5 = clmHMG_allDay5.assign_coords({"pertlim": iPert})
#     clmHMG_allDay6 = clmHMG_allDay6.assign_coords({"pertlim": iPert})
    
#     clmHTG_allDay1 = clmHTG_allDay1.assign_coords({"pertlim": iPert})
    clmHTG_allDay2 = clmHTG_allDay2.assign_coords({"pertlim": iPert})
    clmHTG_allDay3 = clmHTG_allDay3.assign_coords({"pertlim": iPert})
    clmHTG_allDay4 = clmHTG_allDay4.assign_coords({"pertlim": iPert})
    clmHTG_allDay5 = clmHTG_allDay5.assign_coords({"pertlim": iPert})
    clmHTG_allDay6 = clmHTG_allDay6.assign_coords({"pertlim": iPert})
  

    if pertCount==0: 
#         hmgDS_pertDay1 = hmg_allDay1
#         hmgDS_pertDay2 = hmg_allDay2
#         hmgDS_pertDay3 = hmg_allDay3
#         hmgDS_pertDay4 = hmg_allDay4
#         hmgDS_pertDay5 = hmg_allDay5
#         hmgDS_pertDay6 = hmg_allDay6
        
#         htgDS_pertDay1 = htg_allDay1
        htgDS_pertDay2 = htg_allDay2
        htgDS_pertDay3 = htg_allDay3
        htgDS_pertDay4 = htg_allDay4
        htgDS_pertDay5 = htg_allDay5
        htgDS_pertDay6 = htg_allDay6
        
#         clm_hmgDS_pertDay1 = clmHMG_allDay1
#         clm_hmgDS_pertDay2 = clmHMG_allDay2
#         clm_hmgDS_pertDay3 = clmHMG_allDay3
#         clm_hmgDS_pertDay4 = clmHMG_allDay4
#         clm_hmgDS_pertDay5 = clmHMG_allDay5
#         clm_hmgDS_pertDay6 = clmHMG_allDay6
        
#         clm_htgDS_pertDay1 = clmHTG_allDay1
        clm_htgDS_pertDay2 = clmHTG_allDay2
        clm_htgDS_pertDay3 = clmHTG_allDay3
        clm_htgDS_pertDay4 = clmHTG_allDay4
        clm_htgDS_pertDay5 = clmHTG_allDay5
        clm_htgDS_pertDay6 = clmHTG_allDay6
        
        
 
    else: 
#         hmgDS_pertDay1 = xr.concat([hmgDS_pertDay1, hmg_allDay1], "pertlim")
#         hmgDS_pertDay2 = xr.concat([hmgDS_pertDay2, hmg_allDay2], "pertlim")
#         hmgDS_pertDay3 = xr.concat([hmgDS_pertDay3, hmg_allDay3], "pertlim")
#         hmgDS_pertDay4 = xr.concat([hmgDS_pertDay4, hmg_allDay4], "pertlim")
#         hmgDS_pertDay5 = xr.concat([hmgDS_pertDay5, hmg_allDay5], "pertlim")
#         hmgDS_pertDay6 = xr.concat([hmgDS_pertDay6, hmg_allDay6], "pertlim")
        
#         htgDS_pertDay1 = xr.concat([htgDS_pertDay1, htg_allDay1], "pertlim")
        htgDS_pertDay2 = xr.concat([htgDS_pertDay2, htg_allDay2], "pertlim")
        htgDS_pertDay3 = xr.concat([htgDS_pertDay3, htg_allDay3], "pertlim")
        htgDS_pertDay4 = xr.concat([htgDS_pertDay4, htg_allDay4], "pertlim")
        htgDS_pertDay5 = xr.concat([htgDS_pertDay5, htg_allDay5], "pertlim")
        htgDS_pertDay6 = xr.concat([htgDS_pertDay6, htg_allDay6], "pertlim")
        
#         clm_hmgDS_pertDay1 = xr.concat([clm_hmgDS_pertDay1, clmHMG_allDay1], "pertlim")
#         clm_hmgDS_pertDay2 = xr.concat([clm_hmgDS_pertDay2, clmHMG_allDay2], "pertlim")
#         clm_hmgDS_pertDay3 = xr.concat([clm_hmgDS_pertDay3, clmHMG_allDay3], "pertlim")
#         clm_hmgDS_pertDay4 = xr.concat([clm_hmgDS_pertDay4, clmHMG_allDay4], "pertlim")
#         clm_hmgDS_pertDay5 = xr.concat([clm_hmgDS_pertDay5, clmHMG_allDay5], "pertlim")
#         clm_hmgDS_pertDay6 = xr.concat([clm_hmgDS_pertDay6, clmHMG_allDay6], "pertlim")

#         clm_htgDS_pertDay1 = xr.concat([clm_htgDS_pertDay1, clmHTG_allDay1], "pertlim")
        clm_htgDS_pertDay2 = xr.concat([clm_htgDS_pertDay2, clmHTG_allDay2], "pertlim")
        clm_htgDS_pertDay3 = xr.concat([clm_htgDS_pertDay3, clmHTG_allDay3], "pertlim")
        clm_htgDS_pertDay4 = xr.concat([clm_htgDS_pertDay4, clmHTG_allDay4], "pertlim")
        clm_htgDS_pertDay5 = xr.concat([clm_htgDS_pertDay5, clmHTG_allDay5], "pertlim")
        clm_htgDS_pertDay6 = xr.concat([clm_htgDS_pertDay6, clmHTG_allDay6], "pertlim")


#     del hmg_allDay1
#     del hmg_allDay2
#     del hmg_allDay3
#     del hmg_allDay4
#     del hmg_allDay5
#     del hmg_allDay6
    
#     del htg_allDay1
#     del htg_allDay2
#     del htg_allDay3
#     del htg_allDay4
#     del htg_allDay5
#     del htg_allDay6    

    
    print('Done with pertlim ', pertCount)
    
    pertCount=pertCount+1
            

Done with day 0 of 66 
Done with day 1 of 66 
Done with day 2 of 66 
Done with day 3 of 66 
Done with day 4 of 66 
Done with day 5 of 66 
Done with day 6 of 66 
Done with day 7 of 66 
Done with day 8 of 66 
Done with day 9 of 66 
Done with day 10 of 66 
Done with day 11 of 66 
Done with day 12 of 66 
Done with day 13 of 66 
Done with day 14 of 66 
Done with day 15 of 66 
Done with day 16 of 66 
Done with day 17 of 66 
Done with day 18 of 66 
Done with day 19 of 66 
Done with day 20 of 66 
Done with day 21 of 66 
Done with day 22 of 66 
Done with day 23 of 66 
Done with day 24 of 66 
Done with day 25 of 66 
Done with day 26 of 66 
Done with day 27 of 66 
Done with day 28 of 66 
Done with day 29 of 66 
Done with day 30 of 66 
Done with day 31 of 66 
Done with day 32 of 66 
Done with day 33 of 66 
Done with day 34 of 66 
Done with day 35 of 66 
Done with day 36 of 66 
Done with day 37 of 66 
Done with day 38 of 66 
Done with day 39 of 66 
Done with day 40 of 66 
Done with day 41 of 66 
Do

In [7]:
# fig,axs = plt.subplots(1,1, figsize=(18,6) )

# varPlot = 'PBLH'

# plotHMG  = False
# plotDiff = False

# ## Plot just the HMG values 
# if plotHMG==True:
#     axs.plot(hmgDS_pertDay1.time.values, np.squeeze(hmgDS_pertDay1[varPlot].values), color='firebrick',  label='HMG_day1')
#     axs.plot(hmgDS_pertDay2.time.values, np.squeeze(hmgDS_pertDay2[varPlot].values), color='darkorange', label='HMG_day2')
#     axs.plot(hmgDS_pertDay3.time.values, np.squeeze(hmgDS_pertDay3[varPlot].values), color='olivedrab',  label='HMG_day3')
#     axs.plot(hmgDS_pertDay4.time.values, np.squeeze(hmgDS_pertDay4[varPlot].values), color='turquoise',  label='HMG_day4')
#     axs.plot(hmgDS_pertDay5.time.values, np.squeeze(hmgDS_pertDay5[varPlot].values), color='royalblue',  label='HMG_day5')
#     axs.plot(hmgDS_pertDay6.time.values, np.squeeze(hmgDS_pertDay6[varPlot].values), color='blueviolet', label='HMG_day6')
#     axs.set_title('HMG '+varPlot+' (by lead day)')

# ## If you'd rather plot differences....
# if plotDiff==True:
#     axs.plot(hmgDS_pertDay1.time.values, np.squeeze(htgDS_pertDay1[varPlot].values-hmgDS_pertDay1[varPlot].values), color='firebrick',  label='HTG-HMG_day1')
#     axs.plot(hmgDS_pertDay2.time.values, np.squeeze(htgDS_pertDay2[varPlot].values-hmgDS_pertDay2[varPlot].values), color='darkorange', label='HTG-HMG_day2')
#     axs.plot(hmgDS_pertDay3.time.values, np.squeeze(htgDS_pertDay3[varPlot].values-hmgDS_pertDay3[varPlot].values), color='olivedrab',  label='HTG-HMG_day3')
#     axs.plot(hmgDS_pertDay4.time.values, np.squeeze(htgDS_pertDay4[varPlot].values-hmgDS_pertDay4[varPlot].values), color='turquoise',  label='HTG-HMG_day4')
#     axs.plot(hmgDS_pertDay5.time.values, np.squeeze(htgDS_pertDay5[varPlot].values-hmgDS_pertDay5[varPlot].values), color='royalblue',  label='HTG-HMG_day5')
#     axs.plot(hmgDS_pertDay6.time.values, np.squeeze(htgDS_pertDay6[varPlot].values-hmgDS_pertDay6[varPlot].values), color='blueviolet', label='HTG-HMG_day6')
#     axs.set_title('HTG-HMG '+varPlot+' (by lead day)')


# axs.legend()
# axs.set_xlim([datetime.date(2015,6,9), datetime.date(2015,6,25)])


In [7]:
## Convert to local times...
# - - - - - - - - - - - - - - 
# hmgDS_pertDay1_local     = hmgDS_pertDay1.copy(deep=True)
# hmgDS_pertDay2_local     = hmgDS_pertDay2.copy(deep=True)
# hmgDS_pertDay3_local     = hmgDS_pertDay3.copy(deep=True)
# hmgDS_pertDay4_local     = hmgDS_pertDay4.copy(deep=True)
# hmgDS_pertDay5_local     = hmgDS_pertDay5.copy(deep=True)
# hmgDS_pertDay6_local     = hmgDS_pertDay6.copy(deep=True)

# htgDS_pertDay1_local     = htgDS_pertDay1.copy(deep=True)
htgDS_pertDay2_local     = htgDS_pertDay2.copy(deep=True)
htgDS_pertDay3_local     = htgDS_pertDay3.copy(deep=True)
htgDS_pertDay4_local     = htgDS_pertDay4.copy(deep=True)
htgDS_pertDay5_local     = htgDS_pertDay5.copy(deep=True)
htgDS_pertDay6_local     = htgDS_pertDay6.copy(deep=True)

# clm_hmgDS_pertDay1_local = clm_hmgDS_pertDay1.copy(deep=True)
# clm_hmgDS_pertDay2_local = clm_hmgDS_pertDay2.copy(deep=True)
# clm_hmgDS_pertDay3_local = clm_hmgDS_pertDay3.copy(deep=True)
# clm_hmgDS_pertDay4_local = clm_hmgDS_pertDay4.copy(deep=True)
# clm_hmgDS_pertDay5_local = clm_hmgDS_pertDay5.copy(deep=True)
# clm_hmgDS_pertDay6_local = clm_hmgDS_pertDay6.copy(deep=True)

# clm_htgDS_pertDay1_local = clm_htgDS_pertDay1.copy(deep=True)
clm_htgDS_pertDay2_local = clm_htgDS_pertDay2.copy(deep=True)
clm_htgDS_pertDay3_local = clm_htgDS_pertDay3.copy(deep=True)
clm_htgDS_pertDay4_local = clm_htgDS_pertDay4.copy(deep=True)
clm_htgDS_pertDay5_local = clm_htgDS_pertDay5.copy(deep=True)
clm_htgDS_pertDay6_local = clm_htgDS_pertDay6.copy(deep=True)

# Confirmed that all the times are identical, so using the same local time arrays
# localTimes_day1 = htgDS_pertDay1['time'].values - np.timedelta64(5,'h')
localTimes_day2 = htgDS_pertDay2['time'].values - np.timedelta64(5,'h')
localTimes_day3 = htgDS_pertDay3['time'].values - np.timedelta64(5,'h')
localTimes_day4 = htgDS_pertDay4['time'].values - np.timedelta64(5,'h')
localTimes_day5 = htgDS_pertDay5['time'].values - np.timedelta64(5,'h')
localTimes_day6 = htgDS_pertDay6['time'].values - np.timedelta64(5,'h')


# Replace time dimension with local time
# hmgDS_pertDay1_local     = hmgDS_pertDay1_local.assign_coords({"time": localTimes_day1})
# hmgDS_pertDay2_local     = hmgDS_pertDay2_local.assign_coords({"time": localTimes_day2})
# hmgDS_pertDay3_local     = hmgDS_pertDay3_local.assign_coords({"time": localTimes_day3})
# hmgDS_pertDay4_local     = hmgDS_pertDay4_local.assign_coords({"time": localTimes_day4})
# hmgDS_pertDay5_local     = hmgDS_pertDay5_local.assign_coords({"time": localTimes_day5})
# hmgDS_pertDay6_local     = hmgDS_pertDay6_local.assign_coords({"time": localTimes_day6})

# htgDS_pertDay1_local     = htgDS_pertDay1_local.assign_coords({"time": localTimes_day1})
htgDS_pertDay2_local     = htgDS_pertDay2_local.assign_coords({"time": localTimes_day2})
htgDS_pertDay3_local     = htgDS_pertDay3_local.assign_coords({"time": localTimes_day3})
htgDS_pertDay4_local     = htgDS_pertDay4_local.assign_coords({"time": localTimes_day4})
htgDS_pertDay5_local     = htgDS_pertDay5_local.assign_coords({"time": localTimes_day5})
htgDS_pertDay6_local     = htgDS_pertDay6_local.assign_coords({"time": localTimes_day6})

# clm_hmgDS_pertDay1_local = clm_hmgDS_pertDay1_local.assign_coords({"time": localTimes_day1})
# clm_hmgDS_pertDay2_local = clm_hmgDS_pertDay2_local.assign_coords({"time": localTimes_day2})
# clm_hmgDS_pertDay3_local = clm_hmgDS_pertDay3_local.assign_coords({"time": localTimes_day3})
# clm_hmgDS_pertDay4_local = clm_hmgDS_pertDay4_local.assign_coords({"time": localTimes_day4})
# clm_hmgDS_pertDay5_local = clm_hmgDS_pertDay5_local.assign_coords({"time": localTimes_day5})
# clm_hmgDS_pertDay6_local = clm_hmgDS_pertDay6_local.assign_coords({"time": localTimes_day6})

# clm_htgDS_pertDay1_local = clm_htgDS_pertDay1_local.assign_coords({"time": localTimes_day1})
clm_htgDS_pertDay2_local = clm_htgDS_pertDay2_local.assign_coords({"time": localTimes_day2})
clm_htgDS_pertDay3_local = clm_htgDS_pertDay3_local.assign_coords({"time": localTimes_day3})
clm_htgDS_pertDay4_local = clm_htgDS_pertDay4_local.assign_coords({"time": localTimes_day4})
clm_htgDS_pertDay5_local = clm_htgDS_pertDay5_local.assign_coords({"time": localTimes_day5})
clm_htgDS_pertDay6_local = clm_htgDS_pertDay6_local.assign_coords({"time": localTimes_day6})


print('First four times in UTC:\n',   clm_htgDS_pertDay2.time.values[0:5])
print('First four times in local:\n', clm_htgDS_pertDay2_local.time.values[0:5])

## Clear out some memory 
# del hmgDS_pertDay1
# del hmgDS_pertDay2
# del hmgDS_pertDay3
# del hmgDS_pertDay4
# del hmgDS_pertDay5
# del hmgDS_pertDay6

# del htgDS_pertDay1
# del htgDS_pertDay2
# del htgDS_pertDay3
# del htgDS_pertDay4
# del htgDS_pertDay5
# del htgDS_pertDay6

# del clm_hmgDS_pertDay1
# del clm_hmgDS_pertDay2
# del clm_hmgDS_pertDay3
# del clm_hmgDS_pertDay4
# del clm_hmgDS_pertDay5
# del clm_hmgDS_pertDay6

# del clm_htgDS_pertDay1
# del clm_htgDS_pertDay2
# del clm_htgDS_pertDay3
# del clm_htgDS_pertDay4
# del clm_htgDS_pertDay5
# del clm_htgDS_pertDay6


First four times in UTC:
 ['2017-06-03T05:03:30.000000000' '2017-06-03T05:08:30.000000000'
 '2017-06-03T05:13:30.000000000' '2017-06-03T05:18:30.000000000'
 '2017-06-03T05:23:30.000000000']
First four times in local:
 ['2017-06-03T00:03:30.000000000' '2017-06-03T00:08:30.000000000'
 '2017-06-03T00:13:30.000000000' '2017-06-03T00:18:30.000000000'
 '2017-06-03T00:23:30.000000000']


## Look into just a few particular days... 

In [9]:
# def getPBLdefinitions(DS):
#     # So first, get actual pressures 
#     p0       = DS['P0'].values[0]
#     plevm    = DS['hyam']*p0 + DS['hybm']*DS['PS'].isel(lat=0,lon=0) # Mid level
#     plevm.attrs['units'] = "Pa"
    
#      # Get *potential* temperature, not just T 
#     theta = np.asarray(mpc.potential_temperature(plevm * units.pascals, DS['T'] * units.kelvin))
#     # Add to existing DS
#     DS['theta'] = (('time','lev','lat','lon'), theta)

#     # Height with standard atmosphere
#     zlevm_vals = 1000.*np.asarray(mpc.pressure_to_height_std(plevm)) # Units of [m] after multiplied 
#     zlevm      = plevm.copy(deep=True)
#     zlevm[:,:] = zlevm_vals

#     # Now compute the BL depth and save it to the larger CAM dataset
#     PBLdepth = PBLHasMaxDZ(DS, 'theta')
#     print('Done computing PBL depth with theta')

#     PBLdepth_thetal = PBLHasMaxDZ(DS, 'THETAL')
#     print('Done computing PBL depth with THETAL')
    
#     PBLdepth_qAbs = PBLHasMaxDZ_abs(DS,   'Q')
#     print('Done computing PBL depth with Q')
    
#     PBLdepth_thetaAbs = PBLHasMaxDZ_abs(DS, 'theta')
#     print('Done computing PBL depth with theta_abs')
    
#     PBLdepth_thetalAbs = PBLHasMaxDZ_abs(DS, 'THETAL')
#     print('Done computing PBL depth with thetaL_abs')
    
    
#     # Add above to each dataset
#     DS['PBLdepth']    = (('time'), PBLdepth)
#     DS['PBLdepth_Q']  = (('time'), PBLdepth_qAbs)
#     DS['PBLdepth_thetal']  = (('time'), PBLdepth_thetal)
#     DS['PBLdepth_thetaAbs']  = (('time'), PBLdepth_thetaAbs)
#     DS['PBLdepth_thetalAbs']  = (('time'), PBLdepth_thetalAbs)
    
#     ## Belated realization that the heights computed are above *sea level* not above ground level. 
#     #    Need to subtract elevation. 
#     DS['PBLdepth']   = DS['PBLdepth']  - elevation[0][0] 
#     DS['PBLdepth_Q'] = DS['PBLdepth_Q']  - elevation[0][0] 
#     DS['PBLdepth_thetal'] = DS['PBLdepth_thetal']  - elevation[0][0] 
#     DS['PBLdepth_thetaAbs']  = DS['PBLdepth_thetaAbs']  - elevation[0][0] 
#     DS['PBLdepth_thetalAbs'] = DS['PBLdepth_thetalAbs']  - elevation[0][0] 

#     return DS

In [10]:
# ## Read in obs file to correct PBLH elevation (to be above ground level) 
# nateFile  = '/glade/work/mdfowler/CLASP/clasp-htg.bdate.nc'
# nateDS    = xr.open_dataset(nateFile, decode_times=True)
# elevation = nateDS.alt.values

# ## Select the days of interest
# iselTimes = np.where( (hmgDS_pertDay1_local['time.month']==7) & (hmgDS_pertDay1_local['time.day']>=14) & (hmgDS_pertDay1_local['time.day']<=18) )[0]

# # hmgDS_pertDay2_selDays = hmgDS_pertDay2_local.isel(time=iselTimes)
# # htgDS_pertDay2_selDays = htgDS_pertDay2_local.isel(time=iselTimes)

# hmgDS_pertDay2_selDays = hmgDS_pertDay2_local.isel(time=iselTimes).resample(time="30Min").mean()
# htgDS_pertDay2_selDays = htgDS_pertDay2_local.isel(time=iselTimes).resample(time="30Min").mean()

# ## Calculation PBL definitions...
# # hmgDS_pertDay2_withPBLdef = getPBLdefinitions(hmgDS_pertDay2_selDays)
# # htgDS_pertDay2_withPBLdef = getPBLdefinitions(htgDS_pertDay2_selDays)
# hmgDS_pertDay2_withPBLdef_resample = getPBLdefinitions(hmgDS_pertDay2_selDays)
# htgDS_pertDay2_withPBLdef_resample = getPBLdefinitions(htgDS_pertDay2_selDays)


In [11]:
# varNames = ['PBLH','PBLdepth','PBLdepth_Q','PBLdepth_thetal','PBLdepth_thetaAbs','PBLdepth_thetalAbs']

# fig,axs = plt.subplots(len(varNames),1, figsize=(18,4*len(varNames)))
# axs     = axs.ravel()

# for iVar in range(len(varNames)):
#     axs[iVar].plot(hmgDS_pertDay2_withPBLdef_resample.time.values, np.squeeze(hmgDS_pertDay2_withPBLdef_resample[varNames[iVar]].values), 'r', label='HMG' )
#     axs[iVar].plot(htgDS_pertDay2_withPBLdef_resample.time.values, np.squeeze(htgDS_pertDay2_withPBLdef_resample[varNames[iVar]].values), 'b', label='HTG' )

#     axs[iVar].legend()
#     axs[iVar].set_title(varNames[iVar]+' (30m data)')
#     axs[iVar].axvline(datetime.date(2016,7,16), color='grey')
#     axs[iVar].axvline(datetime.date(2016,7,17), color='grey')


In [12]:
# varNames = ['LHFLX','SHFLX','TREFHT','SWCF','LWCF','CLDTOT','CLDLOW','CLDHGH','PRECT']

# fig,axs = plt.subplots(len(varNames),1, figsize=(18,4*len(varNames)))
# axs     = axs.ravel()

# for iVar in range(len(varNames)):
#     axs[iVar].plot(hmgDS_pertDay2_withPBLdef_resample.time.values, np.squeeze(hmgDS_pertDay2_withPBLdef_resample[varNames[iVar]].values), 'r', label='HMG' )
#     axs[iVar].plot(htgDS_pertDay2_withPBLdef_resample.time.values, np.squeeze(htgDS_pertDay2_withPBLdef_resample[varNames[iVar]].values), 'b', label='HTG' )

#     axs[iVar].legend()
#     axs[iVar].set_title(varNames[iVar]+' (30m data)')
#     axs[iVar].axvline(datetime.date(2016,7,16), color='grey')
#     axs[iVar].axvline(datetime.date(2016,7,17), color='grey')
    

In [13]:
# varNames = ['THETAL','theta','Q']

# fig,axs = plt.subplots(len(varNames),2, figsize=(18,4*len(varNames)))
# axs     = axs.ravel()
# fig.subplots_adjust(hspace=0.5)

# nPlot = 0
# for iVar in range(len(varNames)):
#     ## Select just certain levels:
#     iLevs = np.where(hmgDS_pertDay2_withPBLdef_resample.lev.values>=700)[0]
    
#     HMGvar = hmgDS_pertDay2_withPBLdef_resample.isel(lev=iLevs)[varNames[iVar]].values
#     HTGvar = htgDS_pertDay2_withPBLdef_resample.isel(lev=iLevs)[varNames[iVar]].values
    
#     if varNames[iVar]=='Q':
#         levsRaw_min = np.nanmin([np.nanpercentile(HMGvar, 40), np.nanpercentile(HTGvar, 40)])
#         levsRaw_max = np.nanmin([np.nanpercentile(HMGvar, 99), np.nanpercentile(HTGvar, 99)])
#         levsRaw     = np.arange(levsRaw_min, levsRaw_max, (np.abs(levsRaw_max) - np.abs(levsRaw_min))/100.0)
#         cmap_sel = 'viridis'
#     else:
#         levsRaw_min = np.nanmin([np.nanpercentile(HMGvar, 1), np.nanpercentile(HTGvar, 1)])
#         levsRaw_max = np.nanmin([np.nanpercentile(HMGvar, 60), np.nanpercentile(HTGvar, 60)])
#         levsRaw     = np.arange(levsRaw_min, levsRaw_max, (np.abs(levsRaw_max) - np.abs(levsRaw_min))/100.0)
#         cmap_sel = 'plasma'

        
    
#     axs[nPlot].contourf(hmgDS_pertDay2_withPBLdef_resample.time.values, hmgDS_pertDay2_withPBLdef_resample.lev.values[iLevs], 
#                                np.squeeze(HMGvar).transpose(), levsRaw,cmap=cmap_sel, extend='both')
#     cplot=axs[nPlot+1].contourf(htgDS_pertDay2_withPBLdef_resample.time.values, htgDS_pertDay2_withPBLdef_resample.lev.values[iLevs],
#                                np.squeeze(HTGvar).transpose(), levsRaw,cmap=cmap_sel, extend='both')
    
#     # Add colorbar
#     ax_position = axs[nPlot+1].get_position()
#     cbar_ax = fig.add_axes([ax_position.x0-0.2, ax_position.y0-0.06, ax_position.width+0.08, 0.02])
#     if ((varNames[iVar]=='Q') | (varNames[iVar]=='Q_interp')):
#         cbar = plt.colorbar(cplot, orientation='horizontal',cax=cbar_ax,format='%.1e')
#         tick_locator = ticker.MaxNLocator(nbins=5)
#         cbar.locator = tick_locator
#         cbar.update_ticks()
#     else:
#         cbar = plt.colorbar(cplot, orientation='horizontal',cax=cbar_ax)

#     axs[nPlot].set_title(varNames[iVar]+' HMG (30m data)')
#     axs[nPlot+1].set_title(varNames[iVar]+' HTG (30m data)')
    
#     axs[nPlot].axvline(datetime.date(2016,7,16), color='grey')
#     axs[nPlot].axvline(datetime.date(2016,7,17), color='grey')
#     axs[nPlot+1].axvline(datetime.date(2016,7,16), color='grey')
#     axs[nPlot+1].axvline(datetime.date(2016,7,17), color='grey')
    
#     axs[nPlot].invert_yaxis()
#     axs[nPlot+1].invert_yaxis()
    
    
#     nPlot = nPlot+2
    

In [14]:
# varNames = ['PBLH','PBLdepth','PBLdepth_Q','PBLdepth_thetal','PBLdepth_thetaAbs','PBLdepth_thetalAbs']

# fig,axs = plt.subplots(len(varNames),1, figsize=(18,4*len(varNames)))
# axs     = axs.ravel()

# for iVar in range(len(varNames)):
#     axs[iVar].plot(hmgDS_pertDay2_withPBLdef.time.values, np.squeeze(hmgDS_pertDay2_withPBLdef[varNames[iVar]].values), 'r', label='HMG' )
#     axs[iVar].plot(htgDS_pertDay2_withPBLdef.time.values, np.squeeze(htgDS_pertDay2_withPBLdef[varNames[iVar]].values), 'b', label='HTG' )

#     axs[iVar].legend()
#     axs[iVar].set_title(varNames[iVar])
#     axs[iVar].axvline(datetime.date(2016,7,16), color='grey')
#     axs[iVar].axvline(datetime.date(2016,7,17), color='grey')


## Process the data as we've done before

In [8]:
def process_camData(DS):
    ## Add evaporative fraction to DS 
    ds_EF = DS.LHFLX.values / (DS.LHFLX.values + DS.SHFLX.values)
    DS['EvapFraction'] = (('time'), np.squeeze(ds_EF))

    ## Define the actual vertical velocity skew, not just the third order moment 
    skw_W = DS.WP3_CLUBB.values / ((DS.WP2_CLUBB.values)**1.5)
    DS['Skw_W'] = (('time','ilev'), np.squeeze(skw_W))

    ## Add in TKE 
    DS['TKE']  = (('time','ilev','lat,','lon'),
                       0.5*(DS['UP2_CLUBB']+DS['VP2_CLUBB']+DS['WP2_CLUBB'])) 
    DS['TKE'].attrs['units']   = 'm2/s2'
    DS['TKE'].attrs['long_name']   = 'Turbulent Kinetic Energy'
    
    ## Add in wind speed 
    DS['WindMagnitude']  = (('time','lev','lat,','lon'),
                                    np.sqrt((DS.U.values**2.0) + (DS.V.values**2.0)) )

    DS['WindMagnitude'].attrs['units']   = 'm/s'
    DS['WindMagnitude'].attrs['long_name']   = 'Wind speed'
    
    ## Compute PBL as max gradient of theta 
    # - - - - - - - - - - - - - - - - - - - - - - - - - - 
    
    # So first, get actual pressures 
    p0       = DS['P0'].values[0]
    plevm    = DS['hyam']*p0 + DS['hybm']*DS['PS'].isel(lat=0,lon=0) # Mid level
    plevm.attrs['units'] = "Pa"
    
     # Get *potential* temperature, not just T 
    theta = np.asarray(mpc.potential_temperature(plevm * units.pascals, DS['T'] * units.kelvin))
    # Add to existing DS
    DS['theta'] = (('time','lev','lat','lon'), theta)

    # Height with standard atmosphere
    zlevm_vals = 1000.*np.asarray(mpc.pressure_to_height_std(plevm)) # Units of [m] after multiplied 
    zlevm      = plevm.copy(deep=True)
    zlevm[:,:] = zlevm_vals

    # Now compute the BL depth and save it to the larger CAM dataset
    PBLdepth = PBLHasMaxDZ(DS, 'theta')
    print('Done computing PBL depth with theta')

    PBLdepth_qAbs = PBLHasMaxDZ_abs(DS,   'Q')
    print('Done computing PBL depth with Q')
    
    # Add above to each dataset
    DS['PBLdepth']    = (('time'), PBLdepth)
    DS['PBLdepth_Q']  = (('time'), PBLdepth_qAbs)
    
    ## Belated realization that the heights computed are above *sea level* not above ground level. 
    #    Need to subtract elevation. 
    DS['PBLdepth']   = DS['PBLdepth']  - elevation[0][0] 
    DS['PBLdepth_Q'] = DS['PBLdepth_Q']  - elevation[0][0] 

    
    ## Interpolate to standard levels 
    # - - - - - - - - - - - - - - - - - - - - - - - -
    print('Beginning interpolation...') 
    
    # Decide on levels to interpoalte to and add to larger arrays
    pnew64 = np.arange(200.0,980.0,10.0) 

    DS = DS.assign_coords({"levInterp": pnew64})

    varSels = np.asarray(['THLP2_CLUBB','RTP2_CLUBB','RTPTHLP_CLUBB','WPRTP_CLUBB','WPTHLP_CLUBB','WP2_CLUBB','UP2_CLUBB',
                          'VP2_CLUBB','TKE','U','V','T','Q','OMEGA','RVMTEND_CLUBB','STEND_CLUBB','CLOUD','CLOUDFRAC_CLUBB',
                          'UPWP_CLUBB','WP2RTP_CLUBB','THETAL','WindMagnitude'])

    for iVar in range(len(varSels)): 
        varUnits = DS[varSels[iVar]].units
        varName  = DS[varSels[iVar]].long_name

        # Interpolate variables and add to larger arrays 
        interpVar_real = interpolateToPressure_v2(DS, varSels[iVar], pnew64)

        DS[varSels[iVar]+'_interp']  = (('time','levInterp','lat','lon'), interpVar_real)

        ## Assign attibutes 
        DS[varSels[iVar]+'_interp'].attrs['units']     = varUnits
        DS[varSels[iVar]+'_interp'].attrs['long_name'] = varName

    return DS



In [9]:
## Read in obs file to correct PBLH elevation (to be above ground level) 
nateFile  = '/glade/work/mdfowler/CLASP/clasp-htg.bdate.nc'
nateDS    = xr.open_dataset(nateFile, decode_times=True)
elevation = nateDS.alt.values


In [10]:
# ## Process data (HMG)
# hmgDS_local_processed_day1 = process_camData(hmgDS_pertDay1_local)
# hmgDS_local_processed_day2 = process_camData(hmgDS_pertDay2_local)
# hmgDS_local_processed_day3 = process_camData(hmgDS_pertDay3_local)
# hmgDS_local_processed_day4 = process_camData(hmgDS_pertDay4_local)
# hmgDS_local_processed_day5 = process_camData(hmgDS_pertDay5_local)
# hmgDS_local_processed_day6 = process_camData(hmgDS_pertDay6_local)
# print('Done processing HMG DS') 

# htgDS_local_processed_day1 = process_camData(htgDS_pertDay1_local)
htgDS_local_processed_day2 = process_camData(htgDS_pertDay2_local)
htgDS_local_processed_day3 = process_camData(htgDS_pertDay3_local)
htgDS_local_processed_day4 = process_camData(htgDS_pertDay4_local)
htgDS_local_processed_day5 = process_camData(htgDS_pertDay5_local)
htgDS_local_processed_day6 = process_camData(htgDS_pertDay6_local)
print('Done processing HTG DS') 



Done computing PBL depth with theta
Done computing PBL depth with Q
Beginning interpolation...
Done computing PBL depth with theta
Done computing PBL depth with Q
Beginning interpolation...
Done computing PBL depth with theta
Done computing PBL depth with Q
Beginning interpolation...
Done computing PBL depth with theta
Done computing PBL depth with Q
Beginning interpolation...
Done computing PBL depth with theta
Done computing PBL depth with Q
Beginning interpolation...
Done processing HTG DS


In [11]:
## We really don't need all the variables in CLM dataset....
keepVars_CLM = ['SOILWATER_10CM','TSOI_10CM','RAIN','FSA','TG','TSA','USTAR','WSTAR','ZETA']

# clm_hmgDS_pertDay1_local = clm_hmgDS_pertDay1_local[keepVars_CLM]
# clm_hmgDS_pertDay2_local = clm_hmgDS_pertDay2_local[keepVars_CLM]
# clm_hmgDS_pertDay3_local = clm_hmgDS_pertDay3_local[keepVars_CLM]
# clm_hmgDS_pertDay4_local = clm_hmgDS_pertDay4_local[keepVars_CLM]
# clm_hmgDS_pertDay5_local = clm_hmgDS_pertDay5_local[keepVars_CLM]
# clm_hmgDS_pertDay6_local = clm_hmgDS_pertDay6_local[keepVars_CLM]

# clm_htgDS_pertDay1_local = clm_htgDS_pertDay1_local[keepVars_CLM]
clm_htgDS_pertDay2_local = clm_htgDS_pertDay2_local[keepVars_CLM]
clm_htgDS_pertDay3_local = clm_htgDS_pertDay3_local[keepVars_CLM]
clm_htgDS_pertDay4_local = clm_htgDS_pertDay4_local[keepVars_CLM]
clm_htgDS_pertDay5_local = clm_htgDS_pertDay5_local[keepVars_CLM]
clm_htgDS_pertDay6_local = clm_htgDS_pertDay6_local[keepVars_CLM]



In [1]:
# ## Save processed datasets to pickle files for easy read-in 
# saveDir = '/glade/work/mdfowler/CLASP/histData/processedData/ens_byLeadDay/'

# # pickle.dump( hmgDS_local_processed_day1,   open( saveDir+"realSfc_HMG_day1_pert01.p", "wb" ) )
# # pickle.dump( hmgDS_local_processed_day2,   open( saveDir+"realSfc_HMG_day2_pert01.p", "wb" ) )
# # pickle.dump( hmgDS_local_processed_day3,   open( saveDir+"realSfc_HMG_day3_pert01.p", "wb" ) )
# # pickle.dump( hmgDS_local_processed_day4,   open( saveDir+"realSfc_HMG_day4_pert01.p", "wb" ) )
# # pickle.dump( hmgDS_local_processed_day5,   open( saveDir+"realSfc_HMG_day5_pert01.p", "wb" ) )
# # pickle.dump( hmgDS_local_processed_day6,   open( saveDir+"realSfc_HMG_day6_pert01.p", "wb" ) )
# # print('Saved processed HMG files to pickle')

# pickle.dump( htgDS_local_processed_day1,   open( saveDir+"realSfc_HTGmult10_2016day1_pert00.p", "wb" ) )
# pickle.dump( htgDS_local_processed_day2,   open( saveDir+"realSfc_HTGmult10_2016day2_pert00.p", "wb" ) )
# pickle.dump( htgDS_local_processed_day3,   open( saveDir+"realSfc_HTGmult10_2016day3_pert00.p", "wb" ) )
# pickle.dump( htgDS_local_processed_day4,   open( saveDir+"realSfc_HTGmult10_2016day4_pert00.p", "wb" ) )
# pickle.dump( htgDS_local_processed_day5,   open( saveDir+"realSfc_HTGmult10_2016day5_pert00.p", "wb" ) )
# pickle.dump( htgDS_local_processed_day6,   open( saveDir+"realSfc_HTGmult10_2016day6_pert00.p", "wb" ) )
# print('Saved processed HTG files to pickle')

# pickle.dump( clm_htgDS_pertDay1_local,   open( saveDir+"realSfc_HTGclmMult10_2016day1_pert00.p", "wb" ) )
# pickle.dump( clm_htgDS_pertDay2_local,   open( saveDir+"realSfc_HTGclmMult10_2016day2_pert00.p", "wb" ) )
# pickle.dump( clm_htgDS_pertDay3_local,   open( saveDir+"realSfc_HTGclmMult10_2016day3_pert00.p", "wb" ) )
# pickle.dump( clm_htgDS_pertDay4_local,   open( saveDir+"realSfc_HTGclmMult10_2016day4_pert00.p", "wb" ) )
# pickle.dump( clm_htgDS_pertDay5_local,   open( saveDir+"realSfc_HTGclmMult10_2016day5_pert00.p", "wb" ) )
# pickle.dump( clm_htgDS_pertDay6_local,   open( saveDir+"realSfc_HTGclmMult10_2016day6_pert00.p", "wb" ) )
# print('Saved processed HTG files to pickle')


In [12]:
# '''
# Per suggestion here: https://stackoverflow.com/questions/64282393/how-can-i-speed-up-xarray-resample-much-slower-than-pandas-resample 
# Need to check here to get this working better: https://stackoverflow.com/questions/55474088/only-valid-with-datetimeindex-timedeltaindex-or-periodindex-but-got-an-instanc 
# '''
# def quick_resample(ds): 
#     df_h = ds.to_dataframe().set_index('time').resample("30Min").mean()  # what we want (quickly), but in Pandas form
#     vals = [xr.DataArray(data=df_h[c], dims=['time'], coords={'time':df_h.index}, attrs=ds[c].attrs) for c in df_h.columns]
#     ds_h = xr.Dataset(dict(zip(df_h.columns,vals)), attrs=ds.attrs)
    
#     return ds_h 

In [19]:
# df_h = htgDS_local_processed_day1.to_dataframe()

In [ ]:
df_h

In [12]:
## Upsample to 30m? 

# hmg_day1_upsample = hmgDS_local_processed_day1.resample(time="30Min").mean()
# hmg_day2_upsample = hmgDS_local_processed_day2.resample(time="30Min").mean()
# hmg_day3_upsample = hmgDS_local_processed_day3.resample(time="30Min").mean()
# hmg_day4_upsample = hmgDS_local_processed_day4.resample(time="30Min").mean()
# hmg_day5_upsample = hmgDS_local_processed_day5.resample(time="30Min").mean()
# hmg_day6_upsample = hmgDS_local_processed_day6.resample(time="30Min").mean()
# print('Upsampled HMG cases (CAM)')


# htg_day1_upsample = htgDS_local_processed_day1.resample(time="30Min").mean()
htg_day2_upsample = htgDS_local_processed_day2.resample(time="30Min").mean()
htg_day3_upsample = htgDS_local_processed_day3.resample(time="30Min").mean()
htg_day4_upsample = htgDS_local_processed_day4.resample(time="30Min").mean()
htg_day5_upsample = htgDS_local_processed_day5.resample(time="30Min").mean()
htg_day6_upsample = htgDS_local_processed_day6.resample(time="30Min").mean()
print('Upsampled HTG cases (CAM)')


# clm_hmg_day1_upsample = clm_hmgDS_pertDay1_local.resample(time="30Min").mean()
# clm_hmg_day2_upsample = clm_hmgDS_pertDay2_local.resample(time="30Min").mean()
# clm_hmg_day3_upsample = clm_hmgDS_pertDay3_local.resample(time="30Min").mean()
# clm_hmg_day4_upsample = clm_hmgDS_pertDay4_local.resample(time="30Min").mean()
# clm_hmg_day5_upsample = clm_hmgDS_pertDay5_local.resample(time="30Min").mean()
# clm_hmg_day6_upsample = clm_hmgDS_pertDay6_local.resample(time="30Min").mean()
# print('Upsampled HMG cases (CLM)')


# clm_htg_day1_upsample = clm_htgDS_pertDay1_local.resample(time="30Min").mean()
clm_htg_day2_upsample = clm_htgDS_pertDay2_local.resample(time="30Min").mean()
clm_htg_day3_upsample = clm_htgDS_pertDay3_local.resample(time="30Min").mean()
clm_htg_day4_upsample = clm_htgDS_pertDay4_local.resample(time="30Min").mean()
clm_htg_day5_upsample = clm_htgDS_pertDay5_local.resample(time="30Min").mean()
clm_htg_day6_upsample = clm_htgDS_pertDay6_local.resample(time="30Min").mean()
print('Upsampled HTG cases (CLM)')


Upsampled HTG cases (CAM)
Upsampled HTG cases (CLM)


In [13]:
## Save upsampled (30m) datasets to pickle files for easy read-in 
saveDir = '/glade/work/mdfowler/CLASP/histData/processedData/ens_byLeadDay/'

# pickle.dump( hmg_day1_upsample,   open( saveDir+"realSfc_HMG_day1_30min_pert01.p", "wb" ) )
# pickle.dump( hmg_day2_upsample,   open( saveDir+"realSfc_HMG_day2_30min_pert01.p", "wb" ) )
# pickle.dump( hmg_day3_upsample,   open( saveDir+"realSfc_HMG_day3_30min_pert01.p", "wb" ) )
# pickle.dump( hmg_day4_upsample,   open( saveDir+"realSfc_HMG_day4_30min_pert01.p", "wb" ) )
# pickle.dump( hmg_day5_upsample,   open( saveDir+"realSfc_HMG_day5_30min_pert01.p", "wb" ) )
# pickle.dump( hmg_day6_upsample,   open( saveDir+"realSfc_HMG_day6_30min_pert01.p", "wb" ) )
# print('Saved processed/upsampled HMG files to pickle')

# pickle.dump( htg_day1_upsample,   open( saveDir+"realSfc_HTGmult100_2018day1_30min_pert00.p", "wb" ) )
pickle.dump( htg_day2_upsample,   open( saveDir+"realSfc_HTGmult100_2017day2_30min_pert00.p", "wb" ) )
pickle.dump( htg_day3_upsample,   open( saveDir+"realSfc_HTGmult100_2017day3_30min_pert00.p", "wb" ) )
pickle.dump( htg_day4_upsample,   open( saveDir+"realSfc_HTGmult100_2017day4_30min_pert00.p", "wb" ) )
pickle.dump( htg_day5_upsample,   open( saveDir+"realSfc_HTGmult100_2017day5_30min_pert00.p", "wb" ) )
pickle.dump( htg_day6_upsample,   open( saveDir+"realSfc_HTGmult100_2017day6_30min_pert00.p", "wb" ) )
print('Saved processed/upsampled HTG files to pickle')

# pickle.dump( clm_hmg_day1_upsample,   open( saveDir+"realSfc_HMGclm_day1_30min_pert00.p", "wb" ) )
# pickle.dump( clm_hmg_day2_upsample,   open( saveDir+"realSfc_HMGclm_day2_30min_pert00.p", "wb" ) )
# pickle.dump( clm_hmg_day3_upsample,   open( saveDir+"realSfc_HMGclm_day3_30min_pert00.p", "wb" ) )
# pickle.dump( clm_hmg_day4_upsample,   open( saveDir+"realSfc_HMGclm_day4_30min_pert00.p", "wb" ) )
# pickle.dump( clm_hmg_day5_upsample,   open( saveDir+"realSfc_HMGclm_day5_30min_pert00.p", "wb" ) )
# pickle.dump( clm_hmg_day6_upsample,   open( saveDir+"realSfc_HMGclm_day6_30min_pert00.p", "wb" ) )
# print('Saved upsampled CLM_HMG files to pickle')

# pickle.dump( clm_htg_day1_upsample,   open( saveDir+"realSfc_HTGclmMult5_2018day1_30min_pert00.p", "wb" ) )
pickle.dump( clm_htg_day2_upsample,   open( saveDir+"realSfc_HTGclmMult100_2017day2_30min_pert00.p", "wb" ) )
pickle.dump( clm_htg_day3_upsample,   open( saveDir+"realSfc_HTGclmMult100_2017day3_30min_pert00.p", "wb" ) )
pickle.dump( clm_htg_day4_upsample,   open( saveDir+"realSfc_HTGclmMult100_2017day4_30min_pert00.p", "wb" ) )
pickle.dump( clm_htg_day5_upsample,   open( saveDir+"realSfc_HTGclmMult100_2017day5_30min_pert00.p", "wb" ) )
pickle.dump( clm_htg_day6_upsample,   open( saveDir+"realSfc_HTGclmMult100_2017day6_30min_pert00.p", "wb" ) )
print('Saved upsampled CLM_HTG files to pickle')


Saved processed/upsampled HTG files to pickle
Saved upsampled CLM_HTG files to pickle
